In [1]:
import xml.etree.ElementTree as Xet
import pandas as pd

with open('1.full database.xml', 'r', encoding="utf-8") as xml_file:
    xmlparse = Xet.parse(xml_file)

root = xmlparse.getroot()

In [19]:
# *********** extract xml nodes and save to file *****************
tree = Xet.ElementTree(root[5])
tree.write("test.xml", encoding="utf-8")


<class 'xml.etree.ElementTree.Element'>
<class 'xml.etree.ElementTree.Element'>


AttributeError: 'xml.etree.ElementTree.Element' object has no attribute 'tostring'

In [5]:
rows = []
n = 0
for drug in root:
    n += 1
    name = drug.find("{http://www.drugbank.ca}name").text
    drugbankid = drug.find("{http://www.drugbank.ca}drugbank-id").text
    atc_code_elements = drug.findall("{http://www.drugbank.ca}atc-codes/{http://www.drugbank.ca}atc-code")
    targets_elements = drug.findall("{http://www.drugbank.ca}targets/{http://www.drugbank.ca}target/{http://www.drugbank.ca}polypeptide")
    category_elements = drug.findall("{http://www.drugbank.ca}categories/{http://www.drugbank.ca}category")
    drug_interactions_elements = drug.findall("{http://www.drugbank.ca}drug-interactions/{http://www.drugbank.ca}drug-interaction")
    #drug_interaction_drugs = drug.findall("{http://www.drugbank.ca}drug-interactions/{http://www.drugbank.ca}drug-interaction/{http://www.drugbank.ca}name")
    #drug_interaction_descriptions = drug.findall("{http://www.drugbank.ca}drug-interactions/{http://www.drugbank.ca}drug-interaction/{http://www.drugbank.ca}description")
    enzymes_elements = drug.findall("{http://www.drugbank.ca}enzymes/{http://www.drugbank.ca}enzyme")
    #enzymes_uniportId = drug.findall("{http://www.drugbank.ca}enzymes/{http://www.drugbank.ca}enzyme/{http://www.drugbank.ca}polypeptide")
    # for extract smiles
    calculated_properties_elements = drug.findall("{http://www.drugbank.ca}calculated-properties/{http://www.drugbank.ca}property")
    #calculated_value = drug.findall("{http://www.drugbank.ca}calculated-properties/{http://www.drugbank.ca}property/{http://www.drugbank.ca}value")

    
    if len(name) == 0:
        continue
    print(name)
    
    if atc_code_elements is not None:
        atc_codes = "\""
        for atc_code_element in atc_code_elements:
            atc_code = atc_code_element.attrib["code"]
            if len(atc_codes) == 1:
                atc_codes += atc_code
            else:
                atc_codes += "\n" + atc_code
        atc_codes += "\""


    if targets_elements is not None:
        targets = "\""
        for target_element in targets_elements:
            trgt = target_element.attrib["id"]
            if len(targets) == 1:
                targets += trgt
            else:
                targets += "\n" + trgt
        targets += "\""

    categories = "\""
    if category_elements is not None:
        for category_element in category_elements:
            category = category_element.find("{http://www.drugbank.ca}category")
            meshid = category_element.find("{http://www.drugbank.ca}mesh-id")
            #print(category)
            #print(meshid)
            if category is None:
                continue
            else:
                category = category.text
            if meshid is None:
                meshid = ""
            else:
                if meshid.text is None:
                    meshid = ""
                else:
                    meshid = meshid.text
            #print(category)
            #print(meshid)
            if len(categories) == 1:
                categories += category + "/" + meshid
            else:
                categories += "\n" + category + "/" + meshid
    categories += "\""
                
    interactions = "\""
    if drug_interactions_elements is not None:
        for drug_interaction_element in drug_interactions_elements:
            idd = drug_interaction_element.find("{http://www.drugbank.ca}drugbank-id")
            interactionname = drug_interaction_element.find("{http://www.drugbank.ca}name")
            desc = drug_interaction_element.find("{http://www.drugbank.ca}description")
            if idd is None:
                continue
            else:
                idd = idd.text
            if interactionname is None:
                continue
            else:
                interactionname = interactionname.text
            if desc is None:
                desc = ""
            else:
                desc = desc.text
                
            if len(interactions) == 1:
                interactions += idd + "/ " + interactionname + "/ " + desc
            else:

                interactions += "\n" + idd + "/ " + interactionname + "/ " + desc
    interactions += "\""

    enzymes = "\""
    if enzymes_elements is not None:
        for enzyme_element in enzymes_elements:
            enzymename = enzyme_element.find("{http://www.drugbank.ca}name")
            idd = enzyme_element.find("{http://www.drugbank.ca}polypeptide")
            if enzymename is None:
                continue
            else:
                enzymename = enzymename.text
            if idd is None:
                idd = ""
            else:
                idd = idd.attrib["id"]
            if len(enzymes) == 1:
                enzymes += enzymename + "/" + idd
            else:
                enzymes += "\n" + enzymename + "/" + idd
    #print(enzymes)
    enzymes += "\""
    
    smiles = "\""
    if calculated_properties_elements is not None:
        for calculated_element in calculated_properties_elements:
            if calculated_element.find("{http://www.drugbank.ca}kind").text == "SMILES":
                smiles += calculated_element.find("{http://www.drugbank.ca}value").text
    smiles += "\""
    
    rows.append({"drug": name,
                 "drugbank.id": drugbankid,
                "atc-codes": atc_codes,
                "targets": targets,
                "categories": categories,
                "drug_interactions": interactions,
                "enzymes": enzymes,
                "smiles": smiles})
    
    if n % 100 == 0:
        print(n)
print("last one:", n)
del(xmlparse)
del(root)
print("***** making dataframe *****")
df = pd.DataFrame(rows, columns=["drug", "drugbank.id", "atc-codes", "targets", "categories", "drug_interactions", "enzymes", "smiles"])
print("****** writng to file *****")
df.to_csv("output.csv")



Lepirudin
Cetuximab
Dornase alfa
Denileukin diftitox
Etanercept
Bivalirudin
Leuprolide
Peginterferon alfa-2a
Alteplase
Sermorelin
Interferon alfa-n1
Darbepoetin alfa
Urokinase
Goserelin
Reteplase
Erythropoietin
Salmon calcitonin
Interferon alfa-n3
Pegfilgrastim
Sargramostim
Peginterferon alfa-2b
Asparaginase Escherichia coli
Thyrotropin alfa
Antihemophilic factor, human recombinant
Anakinra
Gramicidin D
Human immunoglobulin G
Anistreplase
Insulin human
Tenecteplase
Menotropins
Interferon gamma-1b
Interferon alfa-2a, Recombinant
Desmopressin
Coagulation factor VIIa Recombinant Human
Oprelvekin
Palifermin
Glucagon
Aldesleukin
Botulinum Toxin Type B
Omalizumab
Lutropin alfa
Lyme disease vaccine (recombinant OspA)
Insulin lispro
Insulin glargine
Collagenase clostridium histolyticum
Rasburicase
Cetrorelix
Adalimumab
Somatotropin
Imiglucerase
Abciximab
Drotrecogin alfa
Gemtuzumab ozogamicin
Indium In-111 satumomab pendetide
Alpha-1-proteinase inhibitor
Pegaspargase
Interferon beta-1a
Pegadem

Dextropropoxyphene
Mitotane
Stavudine
Leucovorin
Dyphylline
Pentazocine
Magnesium sulfate
Latanoprost
Estrone
Trazodone
Mecamylamine
Sevelamer
Acamprosate
Metaxalone
Verapamil
Trimethobenzamide
Flumethasone
Sulfametopyrazine
Nilutamide
Nafarelin
Epinephrine
Sumatriptan
Pirenzepine
Cefixime
Chlorpropamide
Aprepitant
Galantamine
Tamoxifen
Benzyl benzoate
Isoflurophate
Losartan
Thioridazine
Moricizine
Amphotericin B
Warfarin
Midazolam
Tobramycin
Trovafloxacin
Pentosan polysulfate
Fludrocortisone
Mycophenolate mofetil
Cephaloglycin
Flurazepam
Moexipril
Phentolamine
Fluorescein
Daunorubicin
Furosemide
Ergotamine
Tizanidine
Nitrofurantoin
Nicergoline
Eplerenone
Amprenavir
Icodextrin
Methazolamide
Naltrexone
Delavirdine
Tamsulosin
Porfimer sodium
Sufentanil
Lamivudine
Ibandronate
Diethylcarbamazine
Flurbiprofen
Oxacillin
Apomorphine
Paroxetine
700
Nedocromil
Norethisterone
Adefovir dipivoxil
Azatadine
Clodronic acid
Procaine
Lisinopril
Methoxamine
Imiquimod
Homatropine methylbromide
Trimipram

Saprisartan
Spirapril
Tasosartan
Amobarbital
Aprobarbital
Butobarbital
Heptabarbital
Hexobarbital
Lithium cation
Mestranol
Penbutolol
Iohexol
Ephedra sinica root
Ephedrine
Mephentermine
Procaterol
Rasagiline
Quinupristin
Aluminium
Calcium
Aluminium monostearate
Magnesium oxide
Magnesium cation
Cortisone acetate
Ginkgo biloba
Glymidine
Paramethasone
Mibefradil
Sodium bicarbonate
Yohimbine
Bezafibrate
Colchicine
Drospirenone
Digitoxin
Magnesium salicylate
Salsalate
Neostigmine
Choline magnesium trisalicylate
Methotrimeprazine
Ginseng
Temafloxacin
Danazol
Clenbuterol
Bambuterol
Tiotropium
Ciclesonide
Pranlukast
Theobromine
Cefepime
Cefacetrile
Ceftibuten
Cefpodoxime
Acenocoumarol
Antrafenine
Testosterone propionate
Paromomycin
Nitroxoline
Stepronin
Aminophenazone
Alizapride
Ajmaline
Amrinone
Oxybenzone
Aprindine
Almitrine
Allylestrenol
Cholestyramine
Methadyl acetate
19-norandrostenedione
Antipyrine
Alfacalcidol
Glutethimide
Phenazopyridine
3-Methylthiofentanyl
gamma-Hydroxybutyric acid
5

[1-(4-Fluorobenzyl)Cyclobutyl]Methyl (1s)-1-[Oxo(1h-Pyrazol-5-Ylamino)Acetyl]Pentylcarbamate
4-CDP-2-C-methyl-D-erythritol 2-phosphate
Cordycepin Triphosphate
Uridine diphosphate glucose
Isopropyl beta-D-thiogalactopyranoside
Inositol 1,3,4,5-Tetrakisphosphate
5'-Guanosine-Diphosphate-Monothiophosphate
3-(6-Aminopyridin-3-Yl)-N-Methyl-N-[(1-Methyl-1h-Indol-2-Yl)Methyl]Acrylamide
RU79256
Glycyl-L-a-Aminopimelyl-E-(D-2-Aminoethyl)Phosphonate
1,4-dithio-alpha-D-glucopyranose
[1-(1-Benzyl-3-Hydroxy-2-Oxo-Propylcarbamoyl)-2-Phenyl-Ethyl]-Carbamic Acid Benzyl Ester
2-deoxy-2-acetamido-beta-D-galactose-4-sulfate
Epothilone D
Tropinone
8-azaxanthine
Bis(5-Amidino-2-Benzimidazolyl)Methanone
N-Hydroxy 1n(4-Methoxyphenyl)Sulfonyl-4-(Z,E-N-Methoxyimino)Pyrrolidine-2r-Carboxamide
Benzophenone
(S)-2-{Methyl-[2-(Naphthalene-2-Sulfonylamino)-5-(Naphthalene-2-Sulfonyloxy)-Benzoyl]-Amino}-Succinicacid
3,4-Dihydroxycinnamic Acid
2-Methylpentane-1,2,4-Triol
(2s)-2-Amino-4-(Methylsulfanyl)-1-Pyridin-2-Ylbu

7-Nitroindazole
Pyridoxine phosphate
Hexane-1,6-Diol
(R)-N-methyl-N-2-propynyl-1-indanamine
Metanitrophenyl-Alpha-D-Galactoside
6,7-dioxo-5H-8-ribitylaminolumazine
Furoyl-Leucine
S-Methylcysteine
Dpb-T
N-[4-hydroxymethyl-cyclohexan-6-yl-1,2,3-triol]-4,6-dideoxy-4-aminoglucopyranoside
2100
3-(cyclohexylamino)propanesulfonic acid
AL7089A
4-(Aminosulfonyl)-N-[(2,4,6-Trifluorophenyl)Methyl]-Benzamide
2,6-Diamino-(S)-9-[2-(Phosphonomethoxy)Propyl]Purine
LY231514 tetra glu
Taxifolin
1,2-Di-N-Pentanoyl-Sn-Glycero-3-Dithiophosphocholine
3,8-Diamino-6-Phenyl-5-[6-[1-[2-[(1,2,3,4-Tetrahydro-9-Acridinyl)Amino]Ethyl]-1h-1,2,3-Triazol-4-Yl]Hexyl]-Phenanthridinium
4,5-Dihydroxy-Tetrahydro-Pyran-2-Carboxylic Acid
2-deoxy-2-fluoro-β-D-galactose
5'-O-[(L-methionyl)-sulphamoyl]adenosine
Immucillin-G
1,2-Dihydroxybenzene
6-hydroxy-D-norleucine
S-Ethylisothiourea
L-methionine (R)-S-oxide
Glycinamide Ribonucleotide
Maltotetraose
4-methylthio-2-oxobutanoic acid
Laevulinic Acid
Quinacrine mustard
8-Benzyl-2-

...

Ferrous sulfate anhydrous
Etofamide
Ethyl chloride
Lysozyme
Sitafloxacin
Aceclidine
Quassia
Penimepicycline
Hexobendine
Cefatrizine
Ritiometan
Acetarsol
Dichlorobenzyl alcohol
Dibekacin
Wood creosote
Meladrazine
Sultopride
Micronomicin
Clorindione
Idanpramine
Benziodarone
Bucetin
Carbocromen
Aluminium acetoacetate
Bromelains
Benzododecinium
Sulfaisodimidine
Meticrane
Etilamfetamine
Bumadizone
Miocamycin
Tromantadine
Rimazolium
Olaflur
Cloridarol
Pimethixene
Ipecac
Azidamfenicol
Atracurium
Propamidine
Idrocilamide
Alsactide
Sulglicotide
Epicillin
Hachimycin
Febarbamate
Triaziquone
Tolpropamine
Chlorquinaldol
Proscillaridin
Oxametacin
Benproperine
Ormeloxifene
Trepibutone
Delapril
Melevodopa
Alminoprofen
Sobrerol
Ibopamine
Flunoxaprofen
Demecolcine
Propenidazole
Sulfaperin
Methiodal
Hydrotalcite
Trichloroethylene
Tetrazepam
Nifurzide
Euflavine
Picotamide
Butanilicaine
Guaiazulen
Amezinium metilsulfate
Pyrithyldione
Dibutylsuccinate
Efloxate
Mannosulfan
Pinazepam
Dimethylphthalate
Pheneti

Plasma protein fraction (human)
Quaternium-24
Dioctyldimonium
Vanadium
Racemethionine
Remestemcel-L
Passiflora incarnata flower
Black cohosh
Sulesomab
Potassium carbonate
Selisistat
Besilesomab
Copper Cu-64
Nomegestrol acetate
Lutetium Lu-177
5,7,2′-trihydroxy-6,8-dimethoxyflavone
Cyclopropane
Lutetium Lu 177 dotatate
Strontium chloride
SB-269970
Epitizide
Dipropyl-4-hydroxytryptamine
Pipequaline
Azintamide
MRK-409
AZD-7325
Ferric pyrophosphate citrate
Magnesium acetate
Baloxavir marboxil
Lonoctocog alfa
Moroctocog alfa
Tetraglyme
alpha-Tocopherol succinate
D-alpha-Tocopherol acetate
alpha-Tocopherol acetate
Tildrakizumab
Dimethicone 410
Choline salicylate
Pentetic acid
Hispidulin
Medical Cannabis
5-methoxy-N,N-dimethyltryptamine
Nabiximols
Burosumab
SR-9009
SR-9011
Sodium bisulfite
Ox bile extract
H3B-8800
Bromotheophylline
Fosnetupitant
Benzoin
Typhoid vaccine
Clinafloxacin
Thiosalicylic acid
Taspoglutide
Nordazepam
Furafylline
PZM21
Tretamine
Acetyl sulfisoxazole
Darglitazone
Englit

Sodium phosphate, dibasic
Sodium phosphate, monobasic, unspecified form
Sodium phosphate, dibasic, unspecified form
Sodium borate
Lithium hydroxide
Lithium citrate
Lithium succinate
Lithium carbonate
Ferrous cation
Acetate
Mometasone furoate
Magnesium
Magnesium levulinate
Magnesium lactate
Sodium cation
Aluminium phosphate
Aluminum acetate
Aluminum cation
Tetraferric tricitrate decahydrate
Silver cation
Hydroxide ion
Phosphate ion
Gallium cation
Chromic cation
Chromic citrate
Chromic nitrate
Chromium gluconate
Chromium nicotinate
Chromous sulfate
Carbonate ion
Zinc cation
Zinc chloride
Gold cation (1+)
Indium cation In-111
Strontium cation Sr-89
Chromate ion Cr-51
Hydrocortisone aceponate
Hydrocortisone acetate
Hydrocortisone butyrate
Hydrocortisone cypionate
Hydrocortisone phosphate
Hydrocortisone probutate
Hydrocortisone valerate
Hydrocortisone succinate
Sulfate ion
Chloride ion
Zinc sulfate, unspecified form
Gallium cation Ga-67
Gallium chloride Ga-67
Phosphorus P-32
Cupric cation
B

Apomab
BMS-791826
Apararenone
Girentuximab I-124
CXA-10
Adriforant
MK-1064
AZD-8186
GTI-2040
Nemorexant
Obexelimab
Flortaucipir
PRI-724
Zanubrutinib
Sitravatinib
Poly(lactic acid)
Litoxetine
ITI-214
TP-271
Florilglutamic acid F-18
Tavolixizumab
Toripalimab
Tafasitamab
Ipafricept
LY-2881835
BMS-919373
Licogliflozin
Neopterin
Ethaselen
Ansofaxine
Lintuzumab satetraxetan AC-225
TAS-117
ABT-639
Bifenthrin
NUC-1031
Flutemetamol
Aprocitentan
Lefitolimod
Inarigivir
Inarigivir soproxil
Teprasiran
PF-06291874
Givosiran
Volanesorsen
Agerafenib
Pexastimogene devacirepvec
GI heptapeptide
Omidenepag isopropyl
Beinaglutide
Ribose
Aglatimagene besadenovec
4-demethyl-4-cholesteryloxycarbonylpenclomedine
Abrilumab
Efinopegdutide
Rogaratinib
Troriluzole
ELX-02
Melanoma antigen recognized by T-cells 1
Cabiralizumab
Domagrozumab
GS-9191
Fulacimstat
Ocrylate
Psicose
Oleclumab
Frovocimab
Tezepelumab
Upadacitinib
LY-2624803
Somapacitan
13000
Anfibatide
Leridistim
18-methoxycoronaridine
Gefapixant
Morinidazol

***** making dataframe *****
****** writng to file *****


In [43]:
with open('sixthNode.xml', 'r', encoding="utf-8") as f:
    pa = Xet.parse(f)

newroot = pa.getroot()

In [52]:
for drug in newroot:

    dr = drug.findall("dr")
    if dr[0].find("dr") is None:
        print("niceeee")
    print(dr[0].find("dr").text)

niceeee


AttributeError: 'NoneType' object has no attribute 'text'